#### Authentification

In [11]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
#sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id='<your client id here>',
#                                                          client_secret='<your client secret here>'))

In [12]:
secrets_file = open("secrets.txt","r")

In [13]:
string = secrets_file.read()

In [14]:
#string

In [15]:
#string.split('\n')

In [19]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

#### Authentification

In [20]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [22]:
#1hMzceeWw7QiI6vaBkcEJO

# Handling large playlists - 10000 Songs

In [23]:
playlist = sp.user_playlist_tracks("spotify", "1hMzceeWw7QiI6vaBkcEJO")

In [24]:
playlist["total"] 

10000

In [25]:
len(playlist["items"])

100

Function to extract all songs from a playlist

In [26]:
playlist['next']

'https://api.spotify.com/v1/playlists/1hMzceeWw7QiI6vaBkcEJO/tracks?offset=100&limit=100&additional_types=track'

In [39]:
#sp.next(playlist)

In [28]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3))
    return tracks

In [31]:
# this will take some seconds to run

all_tracks = get_playlist_tracks("1hMzceeWw7QiI6vaBkcEJO")
len(all_tracks)

10000

# Audio features

In [32]:
# here we look only at the first page
playlist = sp.user_playlist_tracks("spotify", "1hMzceeWw7QiI6vaBkcEJO")

In [33]:
playlist["items"][12]["track"]["name"]

'Somebody to Love Me'

In [34]:
playlist["items"][78]["track"]["name"]

'Red Eyes'

In [35]:
# get the uri of a single song:
song_uri = playlist["items"][12]["track"]["uri"]
song_uri

'spotify:track:2Tky1eOpdLXo8yYawnTMcj'

In [36]:
# get the audio features for that song
sp.audio_features(song_uri)

[{'danceability': 0.694,
  'energy': 0.853,
  'key': 6,
  'loudness': -4.755,
  'mode': 0,
  'speechiness': 0.0415,
  'acousticness': 0.107,
  'instrumentalness': 1.63e-06,
  'liveness': 0.225,
  'valence': 0.566,
  'tempo': 117.896,
  'type': 'audio_features',
  'id': '2Tky1eOpdLXo8yYawnTMcj',
  'uri': 'spotify:track:2Tky1eOpdLXo8yYawnTMcj',
  'track_href': 'https://api.spotify.com/v1/tracks/2Tky1eOpdLXo8yYawnTMcj',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2Tky1eOpdLXo8yYawnTMcj',
  'duration_ms': 297933,
  'time_signature': 4}]

In [37]:
# get the uri of a single song:
song_uri = playlist["items"][78]["track"]["uri"]

In [38]:
# get the audio features for that song
sp.audio_features(song_uri)

[{'danceability': 0.464,
  'energy': 0.815,
  'key': 10,
  'loudness': -6.371,
  'mode': 1,
  'speechiness': 0.0368,
  'acousticness': 0.00871,
  'instrumentalness': 0.201,
  'liveness': 0.162,
  'valence': 0.467,
  'tempo': 161.845,
  'type': 'audio_features',
  'id': '71jGGLe5VtEHjIk5dU2W3S',
  'uri': 'spotify:track:71jGGLe5VtEHjIk5dU2W3S',
  'track_href': 'https://api.spotify.com/v1/tracks/71jGGLe5VtEHjIk5dU2W3S',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/71jGGLe5VtEHjIk5dU2W3S',
  'duration_ms': 298920,
  'time_signature': 4}]